In [1]:
import pandas as pd
import numpy as np

def load_raceline_start(path):
    df = pd.read_csv(path)
    
    # 보통 포맷: t,x,y,yaw,speed 또는 x,y,heading...
    # 너가 준 raceline 파일은 x,y 좌표가 있는 걸로 가정.
    # 가장 앞 row를 start pose로 사용.
    x0 = df.iloc[0]["x"]
    y0 = df.iloc[0]["y"]
    
    return float(x0), float(y0)

raceline_start = load_raceline_start("./raceline_playground.csv")
print("Raceline Start =", raceline_start)

def is_log_matching_raceline(log_path, raceline_start, pos_tol=1.5):
    """
    log_path : RunLogger CSV
    raceline_start: (x0, y0)
    pos_tol : 출발 위치 판단 허용 거리 (m)
    """
    try:
        df = pd.read_csv(log_path)
        # 첫 row
        x = df.iloc[0]["x"]
        y = df.iloc[0]["y"]
    except Exception as e:
        print("[WARN] CSV 파싱 실패:", log_path)
        return False
    
    dx = float(x) - raceline_start[0]
    dy = float(y) - raceline_start[1]
    dist = np.hypot(dx, dy)
    
    return dist <= pos_tol


Raceline Start = (-10.0, 7.475)


In [2]:
import os
import re
import io
import math
import numpy as np
import pandas as pd

log_dir = "./logs/fgm"   # 필요하면 여기만 바꿔

# 파일명 패턴: Opp_bumper..._Gap..._Bub..._SPD..._FGM_..._dur_...s.csv
log_pattern = re.compile(
    r"^Opp_.*"                     # Opp_로 시작
    r"_FFG[-\d.]+"                 # _FFG<fgm_fov_angle>
    r"_SCD[-\d.]+"                 # _SCD<fgm_speed_check_fov_deg>
    r"_Gap[-\d.]+"                 # _Gap<...>
    r"_Bub[-\d.]+"                 # _Bub<...>
    r"_Clr[-\d.]+"                 # _Clr<...>
    r"_WW[-\d.]+"                  # _WW<...>
    r"_AW[-\d.]+"                  # _AW<...>
    r"_SW[-\d.]+"                  # _SW<...>
    r"_HB[-\d.]+"                  # _HB<...>
    r"_CT[-\d.]+"                  # _CT<...>
    r"_SA[-\d.]+"                  # _SA<...>
    r"_DBS[-\d.]+"                 # _DBS<...>
    r"_SPD[-\d.]+"                 # _SPD<...>
    r"_FGM_.*_dur_[-\d.]+s\.csv$"  # _FGM_<...>_dur_<t>s.csv
)

all_files = [
    os.path.join(log_dir, f)
    for f in os.listdir(log_dir)
    if f.endswith(".csv") and log_pattern.match(f)
]

print(f"발견된 로그 파일 개수: {len(all_files)}")
for f in all_files[:5]:
    print(" -", os.path.basename(f))

filtered_files = []
for f in all_files:
    if is_log_matching_raceline(f, raceline_start, pos_tol=1.5):
        filtered_files.append(f)
    else:
        print("❌ [FILTERED] Start position mismatch:", f)

print(f"✓ Valid logs: {len(filtered_files)} / {len(all_files)}")


발견된 로그 파일 개수: 304
 - Opp_bumper_slow_2.csv_FFG196.7415_SCD27.2189_Gap1.1556_Bub0.5343_Clr0.5908_WW0.5388_AW5.4560_SW0.0834_HB3.6392_CT0.4046_SA0.5276_DBS0.1159_SPD4.0507_FGM_1764853363_GOAL_dur_8.83s.csv
 - Opp_bumper_slow_1.csv_FFG190.0_SCD35.0_Gap1.5_Bub0.7_Clr0.75_WW0.4_AW4.0_SW0.13_HB3.0_CT0.35_SA0.3_DBS0.22_SPD3.0_FGM_1764850010_GOAL_dur_19.19s.csv
 - Opp_bumper_v_2.csv_FFG174.3867_SCD22.2466_Gap1.2610_Bub0.4911_Clr0.4853_WW0.5513_AW6.1505_SW0.0854_HB2.3195_CT0.3083_SA0.5166_DBS0.1105_SPD3.3177_FGM_1764853752_GOAL_dur_11.78s.csv
 - Opp_bumper_v_5.csv_FFG163.4341_SCD27.2839_Gap1.1548_Bub0.4744_Clr0.5868_WW0.6718_AW5.5844_SW0.0970_HB3.0288_CT0.3893_SA0.5065_DBS0.1256_SPD4.2276_FGM_1764854553_GOAL_dur_10.52s.csv
 - Opp_bumper_slow_0.5.csv_FFG194.2587_SCD27.3071_Gap1.2439_Bub0.4735_Clr0.5826_WW0.5817_AW5.4260_SW0.0935_HB3.3686_CT0.4004_SA0.5192_DBS0.1243_SPD4.2129_FGM_1764852461_GOAL_dur_12.97s.csv
❌ [FILTERED] Start position mismatch: ./logs/fgm\Opp_bumper_slow_0.5.csv_FFG194.2587_SC

In [3]:
def load_run_log(path: str):
    """
    하나의 run_logger csv 파일을
    - df_data: per-step 데이터 (t,x,y,...)
    - summary_dict: 맨 아래 요약 블록 (key,value)
    로 나눠서 반환.
    """
    with open(path, "r") as f:
        lines = f.readlines()
    
    # Summary 시작 라인 찾기
    summary_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith('--- Experiment Summary'):
            summary_idx = i
            break
    
    if summary_idx is None:
        # Summary 없는 경우 (비상용)
        data_text = "".join(lines)
        df = pd.read_csv(io.StringIO(data_text))
        summary = {}
    else:
        # 맨 위 ~ summary 직전 빈 줄까지만 데이터로 사용
        data_text = "".join(lines[:summary_idx-1])
        df = pd.read_csv(io.StringIO(data_text))
        
        # summary 부분은 "key,value" 형식
        summary_lines = [l.strip() for l in lines[summary_idx+1:] if l.strip()]
        summary = {}
        for l in summary_lines:
            if "," in l:
                k, v = l.split(",", 1)
                summary[k.strip()] = v.strip()
    
    return df, summary


In [4]:
def parse_params_from_filename(path: str):
    """
    파일 이름에서
      scenario, Gap, Bub, Clr, WW, AW, SW, HB, CT, SA, DBS, SPD, result, duration_from_name
    를 뽑아서 dict로 반환.
    """
    name = os.path.basename(path)
    
    # 파라미터 블럭 파싱
    param_pattern = re.compile(
        r"_FFG(?P<FFG>[-\d.]+)"
        r"_SCD(?P<SCD>[-\d.]+)"
        r"_Gap(?P<Gap>[-\d.]+)"
        r"_Bub(?P<Bub>[-\d.]+)"
        r"_Clr(?P<Clr>[-\d.]+)"
        r"_WW(?P<WW>[-\d.]+)"
        r"_AW(?P<AW>[-\d.]+)"
        r"_SW(?P<SW>[-\d.]+)"
        r"_HB(?P<HB>[-\d.]+)"
        r"_CT(?P<CT>[-\d.]+)"
        r"_SA(?P<SA>[-\d.]+)"
        r"_DBS(?P<DBS>[-\d.]+)"
        r"_SPD(?P<SPD>[-\d.]+)"
    )

    
    m = param_pattern.search(name)
    params = {}
    if m:
        for k, v in m.groupdict().items():
            try:
                params[k] = float(v)
            except ValueError:
                params[k] = None
    
    # scenario (Gap 앞부분 전체)
    scen = name.split("_Gap")[0]
    params["scenario"] = scen   # ex) Opp_bumper_slow_0.5.csv
    
    # 결과 태그 (GOAL / CRASH / STUCK / ABORT 등)
    m2 = re.search(r"_(GOAL|CRASH|STUCK|ABORT)_", name)
    if m2:
        params["result"] = m2.group(1)
    
    # 파일명에 있는 duration
    m3 = re.search(r"_dur_([0-9.]+)s\.csv$", name)
    if m3:
        try:
            params["duration_from_name"] = float(m3.group(1))
        except ValueError:
            params["duration_from_name"] = None
    
    return params


In [5]:
def parse_summary_types(summary: dict):
    """
    summary dict 의 value를 적당히 숫자/불리언으로 캐스팅.
    """
    out = {}
    for k, v in summary.items():
        v = v.strip()
        # 숫자 시도
        try:
            out[k] = float(v)
            continue
        except ValueError:
            pass
        
        # YES/NO -> 1/0
        if v.upper() in ("YES", "NO"):
            out[k] = 1 if v.upper() == "YES" else 0
        else:
            out[k] = v
    return out


In [6]:
def compute_metrics(df: pd.DataFrame, safe_dist: float = 0.5):
    """
    run_logger 데이터 df 에서
    - total_time
    - collision_flag
    - min_dist / unsafe_ratio
    - fgm_min_dist / fgm_unsafe_ratio / fgm_track_rms
    를 계산 (있으면).
    """
    metrics = {}
    
    # 총 시간
    if "t" in df.columns and len(df) > 1:
        metrics["total_time"] = float(df["t"].iloc[-1] - df["t"].iloc[0])
    else:
        metrics["total_time"] = 0.0
    
    # 충돌 플래그
    if "collision" in df.columns:
        metrics["collision_flag"] = int(df["collision"].max() > 0)
    else:
        metrics["collision_flag"] = 0
    
    # 전역 min_d / unsafe_ratio
    if "min_d" in df.columns:
        metrics["min_dist"] = float(df["min_d"].min())
        metrics["unsafe_ratio"] = float((df["min_d"] < safe_dist).mean())
    
    # FGM 전용
    if "planner" in df.columns and "fgm_min_d" in df.columns:
        mask_fgm = df["planner"].astype(str).str.contains("FGM")
        if mask_fgm.any():
            df_fgm = df[mask_fgm]
            metrics["fgm_min_dist"] = float(df_fgm["fgm_min_d"].min())
            metrics["fgm_unsafe_ratio"] = float((df_fgm["fgm_min_d"] < safe_dist).mean())
            if "fgm_track" in df_fgm.columns:
                metrics["fgm_track_rms"] = float(math.sqrt((df_fgm["fgm_track"] ** 2).mean()))
    
    return metrics


In [7]:
def analyze_log(path: str, safe_dist: float = 0.5):
    """
    하나의 로그 파일 경로를 받아서
    - 파일 이름 기반 파라미터
    - summary 블럭 내용
    - per-step metrics
    다 합쳐서 dict 로 리턴.
    """
    df, summary = load_run_log(path)
    params   = parse_params_from_filename(path)
    metrics  = compute_metrics(df, safe_dist=safe_dist)
    summary_parsed = parse_summary_types(summary)
    
    row = {}
    row.update(params)
    row.update(summary_parsed)
    row.update(metrics)
    row["log_path"] = path
    
    return row


In [8]:
rows = []
for p in filtered_files:
    row = analyze_log(p, safe_dist=0.5)
    rows.append(row)

summary_df = pd.DataFrame(rows)
print("정리된 행 개수:", len(summary_df))
summary_df.head()


정리된 행 개수: 266


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,...,Min Dist (m),Unsafe Ratio,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),total_time,collision_flag,min_dist,unsafe_ratio,log_path
0,196.7415,27.2189,1.1556,0.5343,0.5908,0.5388,5.4560,0.0834,3.6392,0.4046,...,0.0,1.0,1.000000e+09,0.0,0.0,8.832150,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_2.csv_FFG196.7415_S...
1,190.0000,35.0000,1.5000,0.7000,0.7500,0.4000,4.0000,0.1300,3.0000,0.3500,...,0.0,1.0,1.000000e+09,0.0,0.0,19.191206,0,0.0,1.0,./logs/fgm\Opp_bumper_slow_1.csv_FFG190.0_SCD3...
2,174.3867,22.2466,1.2610,0.4911,0.4853,0.5513,6.1505,0.0854,2.3195,0.3083,...,0.0,1.0,1.000000e+09,0.0,0.0,11.784627,0,0.0,1.0,./logs/fgm\Opp_bumper_v_2.csv_FFG174.3867_SCD2...
3,163.4341,27.2839,1.1548,0.4744,0.5868,0.6718,5.5844,0.0970,3.0288,0.3893,...,0.0,1.0,1.000000e+09,0.0,0.0,10.520024,0,0.0,1.0,./logs/fgm\Opp_bumper_v_5.csv_FFG163.4341_SCD2...
4,179.4130,23.6904,1.2428,0.4818,0.4961,0.5387,6.0828,0.0901,2.0256,0.2476,...,0.0,1.0,1.000000e+09,0.0,0.0,11.159615,0,0.0,1.0,./logs/fgm\Opp_bumper_v_5.csv_FFG179.4130_SCD2...


In [9]:
def get_first_existing(row, candidates, default=np.nan):
    for c in candidates:
        if c in row and not pd.isna(row[c]):
            return row[c]
    return default


In [10]:
def compute_safety_comfort_score(row,
                                 d_safe=0.5,
                                 d_good=0.8,
                                 track_rms_bad=1.0,
                                 w_safety=0.6,
                                 w_progress=0.3,
                                 w_comfort=0.1):
    """
    progress는 아직 모르는 상태라 safety+comfort만 보고 임시 점수 계산.
    (progress는 나중에 duration min/max로 정규화해서 반영)
    """
    # 충돌 / 비정상 종료 체크
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # ----- Safety -----
    fgm_min = get_first_existing(row, ["FGM Min Dist (m)", "fgm_min_dist", "Min Dist (m)", "min_dist"], default=np.nan)
    if math.isnan(fgm_min):
        fgm_min = d_safe
    
    fgm_unsafe = get_first_existing(row, ["FGM Unsafe Ratio", "fgm_unsafe_ratio", "Unsafe Ratio", "unsafe_ratio"], default=0.0)
    if math.isnan(fgm_unsafe):
        fgm_unsafe = 0.0
    
    # 최소 거리 점수
    Sd = (fgm_min - d_safe) / (d_good - d_safe) if d_good > d_safe else 1.0
    Sd = max(0.0, min(1.0, Sd))
    
    # Unsafe Ratio 점수
    Su = 1.0 - max(0.0, min(1.0, fgm_unsafe))
    
    S_safety = 0.7 * Sd + 0.3 * Su
    
    # ----- Comfort (트랙 RMS) -----
    track_rms = get_first_existing(row, ["FGM Track RMS (m)", "fgm_track_rms"], default=0.0)
    if math.isnan(track_rms):
        track_rms = 0.0
    
    S_track = 1.0 - max(0.0, min(1.0, track_rms / track_rms_bad))
    S_comfort = S_track
    
    # progress 제외 상태에서 임시 점수 (safety+comfort 비율만 유지)
    # w_progress는 일단 무시하고, (w_safety + w_comfort)만 사용
    w_sc = w_safety + w_comfort
    Score0 = 0.0
    if w_sc > 0:
        Score0 = (w_safety * S_safety + w_comfort * S_comfort) / w_sc
    return Score0 * 100.0


In [11]:
# 1) safety/comfort 기반 임시 점수 계산
summary_df["Score_tmp"] = summary_df.apply(compute_safety_comfort_score, axis=1)

# 2) duration_raw 칼럼 준비 (Summary > 파일명 순으로)
summary_df["duration_raw"] = summary_df.apply(
    lambda r: get_first_existing(r, ["Duration (s)", "duration_from_name"], default=np.nan),
    axis=1
)

# 3) duration min/max (충돌/ABORT 제외)
valid_mask = (summary_df["Score_tmp"] > 0) & (~summary_df["duration_raw"].isna())
if valid_mask.any():
    T_min = summary_df.loc[valid_mask, "duration_raw"].min()
    T_max = summary_df.loc[valid_mask, "duration_raw"].max()
else:
    T_min, T_max = np.nan, np.nan

def compute_progress(duration, T_min, T_max):
    if math.isnan(duration) or math.isnan(T_min) or math.isnan(T_max) or T_max <= T_min:
        return 0.0
    val = (T_max - duration) / (T_max - T_min)
    return max(0.0, min(1.0, val))

summary_df["S_progress"] = summary_df["duration_raw"].apply(
    lambda t: compute_progress(t, T_min, T_max)
)

def compute_final_score(row,
                        w_safety=0.6,
                        w_progress=0.3,
                        w_comfort=0.1):
    # 충돌 / 비정상 종료면 0점
    result = str(row.get("result", "")).upper()
    collision_flag = 0
    if "Collision" in row:
        collision_flag = int(row["Collision"])
    elif "collision_flag" in row:
        collision_flag = int(row["collision_flag"])
    if collision_flag == 1 or result in ("CRASH", "STUCK", "ABORT"):
        return 0.0
    
    # safety+comfort는 Score_tmp 에 이미 들어 있음 (0~100)
    Score_tmp = row["Score_tmp"]
    # progress는 0~1
    S_progress = row["S_progress"]
    
    # 안전+컴포트 점수 비율 = (1 - w_progress)
    # 최종: Score = (1 - w_progress)*Score_tmp + w_progress*(100 * S_progress)
    Score_final = (1.0 - w_progress) * Score_tmp + w_progress * (100.0 * S_progress)
    
    # 클립
    return max(0.0, min(100.0, Score_final))

summary_df["Score"] = summary_df.apply(compute_final_score, axis=1)

# 점수 높은 순으로 보기
summary_df_sorted = summary_df.sort_values("Score", ascending=False)
summary_df_sorted.head(10)[[
    "scenario", "result",
    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD",
    "Score",
    "FGM Min Dist (m)","FGM Unsafe Ratio","FGM Track RMS (m)",
    "Duration (s)","duration_from_name"
]]


,scenario,result,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,...,CT,SA,DBS,SPD,Score,FGM Min Dist (m),FGM Unsafe Ratio,FGM Track RMS (m),Duration (s),duration_from_name
0,Opp_bumper_slow_2.csv_FFG196.7415_SCD27.2189,GOAL,196.7415,27.2189,1.1556,0.5343,0.5908,0.5388,5.4560,0.0834,...,0.4046,0.5276,0.1159,4.0507,100.000000,1.000000e+09,0.0,0.0,8.83,8.83
66,Opp_bumper_slow_2.csv_FFG181.8760_SCD26.9655,GOAL,181.8760,26.9655,1.1521,0.4526,0.5075,0.5652,7.0217,0.0747,...,0.2319,0.5373,0.1237,3.9499,99.499755,1.000000e+09,0.0,0.0,9.17,9.17
57,Opp_bumper_v_2.csv_FFG190.5232_SCD21.8559,GOAL,190.5232,21.8559,0.9946,0.3625,0.4313,0.7846,8.4080,0.0472,...,0.2005,0.5416,0.1703,5.6373,98.602256,1.000000e+09,0.0,0.0,9.78,9.78
136,Opp_bumper_v_3.csv_FFG190.5232_SCD21.8559,GOAL,190.5232,21.8559,0.9946,0.3625,0.4313,0.7846,8.4080,0.0472,...,0.2005,0.5416,0.1703,5.6373,98.602256,1.000000e+09,0.0,0.0,9.78,9.78
28,Opp_bumper_slow_1.5.csv_FFG183.7975_SCD25.1049,GOAL,183.7975,25.1049,0.9766,0.3967,0.4409,0.7085,8.9418,0.0397,...,0.2028,0.6142,0.1746,4.7270,98.602256,1.000000e+09,0.0,0.0,9.78,9.78
70,Opp_bumper_v_4.csv_FFG151.1493_SCD28.2085,GOAL,151.1493,28.2085,0.9634,0.4035,0.4971,0.6103,7.1989,0.0436,...,0.2070,0.6252,0.1840,4.8784,98.587543,1.000000e+09,0.0,0.0,9.79,9.79
107,Opp_bumper_v_2.csv_FFG183.7975_SCD25.1049,GOAL,183.7975,25.1049,0.9766,0.3967,0.4409,0.7085,8.9418,0.0397,...,0.2028,0.6142,0.1746,4.7270,98.587543,1.000000e+09,0.0,0.0,9.79,9.79
246,Opp_bumper_slow_2.csv_FFG190.5232_SCD21.8559,GOAL,190.5232,21.8559,0.9946,0.3625,0.4313,0.7846,8.4080,0.0472,...,0.2005,0.5416,0.1703,5.6373,98.587543,1.000000e+09,0.0,0.0,9.79,9.79
179,Opp_bumper_v_5.csv_FFG190.5232_SCD21.8559,GOAL,190.5232,21.8559,0.9946,0.3625,0.4313,0.7846,8.4080,0.0472,...,0.2005,0.5416,0.1703,5.6373,98.572830,1.000000e+09,0.0,0.0,9.80,9.80
218,Opp_bumper_slow_1.5.csv_FFG151.1493_SCD28.2085,GOAL,151.1493,28.2085,0.9634,0.4035,0.4971,0.6103,7.1989,0.0436,...,0.2070,0.6252,0.1840,4.8784,98.558117,1.000000e+09,0.0,0.0,9.81,9.81


In [12]:
param_cols = [    "FFG", "SCD", "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"]

grouped = (
    summary_df
    .groupby(param_cols)
    .agg(
        mean_score=("Score", "mean"),
        min_score=("Score", "min"),
        max_score=("Score", "max"),
        runs=("Score", "count"),
        crashes=("Collision", "sum") if "Collision" in summary_df.columns else ("collision_flag", "sum")
    )
    .reset_index()
    .sort_values("mean_score", ascending=False)
)

grouped


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD,mean_score,min_score,max_score,runs,crashes
27,190.5232,21.8559,0.9946,0.3625,0.4313,0.7846,8.4080,0.0472,1.3784,0.2005,0.5416,0.1703,5.6373,98.550760,98.440412,98.602256,8,0
35,205.1127,25.8780,1.0662,0.3905,0.4524,0.6609,8.6333,0.0453,1.5939,0.1969,0.6562,0.1908,6.0000,98.502943,98.410986,98.543404,8,0
22,184.7231,28.4547,0.8773,0.4276,0.4382,0.6966,7.3261,0.0487,1.6560,0.2093,0.6832,0.1970,5.6932,98.495586,98.425699,98.528691,8,0
11,178.1999,27.9629,0.9405,0.3737,0.4573,0.7473,8.0073,0.0553,1.6028,0.2135,0.6266,0.2135,5.4097,98.490857,98.425699,98.528691,7,0
10,176.5703,24.4672,1.0186,0.3822,0.4670,0.7567,8.0995,0.0514,1.5064,0.2000,0.5654,0.1846,5.4572,98.488230,98.307994,98.558117,8,0
20,183.7975,25.1049,0.9766,0.3967,0.4409,0.7085,8.9418,0.0397,1.4716,0.2028,0.6142,0.1746,4.7270,98.488230,98.205002,98.602256,8,0
32,196.6365,26.2175,0.8167,0.4097,0.4526,0.7232,9.0344,0.0417,1.4291,0.2095,0.5241,0.2013,5.6621,98.486390,98.410986,98.543404,8,0
0,151.1493,28.2085,0.9634,0.4035,0.4971,0.6103,7.1989,0.0436,1.4047,0.2070,0.6252,0.1840,4.8784,98.480873,98.381560,98.587543,8,0
18,183.4920,25.3531,1.0959,0.5036,0.5600,0.7208,6.9009,0.0745,1.9540,0.2207,0.4764,0.1230,4.3859,98.474042,98.337420,98.528691,7,0
28,192.1908,23.8581,1.0651,0.4342,0.4584,0.7131,8.1730,0.0465,1.4823,0.1976,0.6184,0.1944,5.0342,98.440412,98.146150,98.558117,8,0


In [13]:
import numpy as np
import pandas as pd

# FFG = fgm_fov_angle
# SCD = fgm_speed_check_fov_deg
param_cols = [
    "FFG", "SCD",   # 새로 추가된 두 파라미터
    "Gap", "Bub", "Clr", "WW", "AW", "SW", "HB", "CT", "SA", "DBS", "SPD"
]

# 각 파라미터 튜닝 범위 (대략 이전에 쓰던 값 기준으로 적당히 설정)
param_bounds = {
    "FFG": (140.0, 210.0),   # fov angle [deg]
    "SCD": (10.0,  45.0),    # speed check fov [deg]

    "Gap": (0.8,  2.0),
    "Bub": (0.3,  0.8),
    "Clr": (0.4,  0.8),
    "WW":  (0.2,  1.0),
    "AW":  (2.0, 10.0),
    "SW":  (0.02, 0.2),
    "HB":  (0.0,  4.0),
    "CT":  (0.15, 0.5),
    "SA":  (0.2,  0.8),
    "DBS": (0.05, 0.3),
    "SPD": (2.5,  6.0),
}


In [14]:
rng = np.random.default_rng(42)  # 필요하면 42 말고 다른 숫자로 바꿔도 됨

def propose_new_params_simple(
    grouped: pd.DataFrame,
    param_cols,
    param_bounds,
    n_seeds=3,       # mean_score 상위 몇 세트를 seed로 쓸지
    per_seed=10,     # seed 하나당 몇 개 후보를 생성할지
    noise_scale=0.08 # ±8% 정도 multiplicative noise
):
    """
    grouped: FFG,SCD,Gap~SPD, mean_score 등이 들어있는 DataFrame
    param_cols: 튜닝할 파라미터 컬럼 이름 리스트
    param_bounds: 각 파라미터 별 (min,max) 딕셔너리
    """
    # 1) mean_score 높은 순서대로 상위 n_seeds 뽑기
    top = grouped.sort_values("mean_score", ascending=False).head(n_seeds)
    
    new_rows = []
    for _, seed in top.iterrows():
        base = seed[param_cols].astype(float).values  # seed 파라미터 벡터
        
        for _ in range(per_seed):
            # multiplicative gaussian noise: θ' = θ * (1 + N(0, noise_scale))
            noise = rng.normal(loc=0.0, scale=noise_scale, size=len(param_cols))
            cand = base * (1.0 + noise)
            
            # 각 파라미터 별 bounds 안으로 자르기
            cand_clipped = []
            for val, name in zip(cand, param_cols):
                lo, hi = param_bounds[name]
                cand_clipped.append(float(np.clip(val, lo, hi)))
            
            new_rows.append(dict(zip(param_cols, cand_clipped)))
    
    new_df = pd.DataFrame(new_rows)
    
    # 2) 이미 실험한 파라미터 세트와 완전히 같은 조합은 제거 (중복 방지)
    merged = new_df.merge(grouped[param_cols], on=param_cols, how="left", indicator=True)
    new_unique = merged[merged["_merge"] == "left_only"][param_cols].reset_index(drop=True)
    
    return new_unique


In [15]:
new_candidates = propose_new_params_simple(
    grouped,
    param_cols=param_cols,
    param_bounds=param_bounds,
    n_seeds=3,      # mean_score 상위 3개를 seed로
    per_seed=10,    # seed 하나당 10개씩 → 총 30개 후보
    noise_scale=0.08
)

print("새로 제안된 파라미터 개수:", len(new_candidates))
new_candidates.head()


새로 제안된 파라미터 개수: 30


,FFG,SCD,Gap,Bub,Clr,WW,AW,SW,HB,CT,SA,DBS,SPD
0,195.167654,20.037517,1.054312,0.389776,0.400000,0.702865,8.493991,0.046006,1.376547,0.186817,0.579703,0.180897,5.667079
1,207.704448,22.673327,0.926228,0.373194,0.400000,0.839739,8.374418,0.046502,1.303313,0.220110,0.534905,0.164464,5.478493
2,198.636580,22.494869,1.027440,0.374994,0.505195,0.759090,8.063445,0.044127,1.446325,0.218609,0.536663,0.158854,5.265472
3,200.439442,23.155459,1.037818,0.343200,0.439310,0.791924,8.555099,0.050491,1.403056,0.211390,0.544528,0.174239,5.922001
4,168.313441,21.296964,0.957173,0.343973,0.421806,0.878434,7.825607,0.050856,1.192827,0.195128,0.548652,0.178287,5.958052


In [16]:
def df_to_params_list_strings(df):
    """
    new_candidates DataFrame(FFG,SCD,Gap~SPD 13개) → bash params_list용 문자열 리스트
    """
    lines = []
    for _, row in df.iterrows():
        vals = [
            row["Gap"],
            row["Bub"],
            row["FFG"],
            row["SCD"],
            row["Clr"],
            row["WW"],
            row["AW"],
            row["SW"],
            row["HB"],
            row["CT"],
            row["SA"],
            row["DBS"],
            row["SPD"],
            
        ]
        s = " ".join(f"{v:.4f}" for v in vals)
        lines.append(f"\"{s}\"")
    return lines

param_strings = df_to_params_list_strings(new_candidates)

print("params_list=(")
for s in param_strings:
    print("    " + s)
print(")")


params_list=(
    "1.0543 0.3898 195.1677 20.0375 0.4000 0.7029 8.4940 0.0460 1.3765 0.1868 0.5797 0.1809 5.6671"
    "0.9262 0.3732 207.7044 22.6733 0.4000 0.8397 8.3744 0.0465 1.3033 0.2201 0.5349 0.1645 5.4785"
    "1.0274 0.3750 198.6366 22.4949 0.5052 0.7591 8.0634 0.0441 1.4463 0.2186 0.5367 0.1589 5.2655"
    "1.0378 0.3432 200.4394 23.1555 0.4393 0.7919 8.5551 0.0505 1.4031 0.2114 0.5445 0.1742 5.9220"
    "0.9572 0.3440 168.3134 21.2970 0.4218 0.8784 7.8256 0.0509 1.1928 0.1951 0.5487 0.1783 5.9581"
    "0.9578 0.3874 202.6153 21.2462 0.4247 0.7045 7.6457 0.0437 1.4332 0.2028 0.5715 0.1645 5.7088"
    "1.0309 0.3433 200.0584 21.3150 0.4188 0.7606 7.6036 0.0490 1.3266 0.2007 0.5624 0.1764 5.9374"
    "0.9883 0.3136 189.0221 21.1158 0.4000 0.7016 7.7372 0.0487 1.2786 0.1944 0.5979 0.1654 5.9699"
    "0.9190 0.3527 176.2931 21.4967 0.4603 0.6762 8.7002 0.0481 1.3129 0.1773 0.5447 0.1631 5.7422"
    "0.9756 0.3328 190.8563 24.6566 0.4375 0.7984 9.3222 0.0504 1.4178 0.2240 0.4901 0